In [ ]:
# declare a list tasks whose products you want to use as inputs
upstream = ['train_model']
product = None

<a href="https://colab.research.google.com/github/monimiller/womens_wc_23/blob/main/run_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 7.3 MB/s eta 0:00:00


In [2]:
# Basics
import pandas as pd
import numpy as np
# Viz
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
#Catboost
from catboost import CatBoostClassifier, CatBoostRegressor

In [3]:
!wget https://github.com/monimiller/womens_wc_23/raw/main/models/catboost_model_20230717_2123.cbm

--2023-07-18 02:36:19--  https://github.com/monimiller/womens_wc_23/raw/main/models/catboost_model_20230717_2123.cbm
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/monimiller/womens_wc_23/main/models/catboost_model_20230717_2123.cbm [following]
--2023-07-18 02:36:19--  https://raw.githubusercontent.com/monimiller/womens_wc_23/main/models/catboost_model_20230717_2123.cbm
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21314844 (20M) [application/octet-stream]
Saving to: ‘catboost_model_20230717_2123.cbm’

catboost_model_2023 100%[===================>]  20.33M  41.5MB/s    in 0.5s    

2023-07-18

In [10]:
model = CatBoostClassifier().load_model(upstream['train_model']['model'])
teams_performance = pd.read_parquet(upstream['train_model']['teams_performance'])

In [20]:
group_a = pd.DataFrame({
                      'home_team': ['New Zealand', 'Philippines', 'New Zealand', 'Switzerland'],
                      'away_team': ['Norway', 'Switzerland', 'Philippines', 'Norway']
                      })

group_b = pd.DataFrame({
                      'home_team': ['Zambia', 'Philippines', 'Zambia', 'Switzerland'],
                      'away_team': ['Norway', 'Switzerland', 'Philippines', 'Norway']
                      })


In [6]:
def encode_variables(df):
  "Input a dataframe and return a dataframe whith categorical variables encoded numericaly"
  # Find categorical vars
  cat_vars = df.select_dtypes(include=['object', 'category']).columns
  # Loop and transform
  for variable in cat_vars:
    df[variable] = df[variable].factorize()[0]

  return df

In [22]:
groups = [group_a, group_b]

def make_predictions(group_df):
    X_input = (
    group_df
    .merge(teams_performance, left_on= 'home_team', right_on='team', how='left').fillna(0)
    .merge(teams_performance, left_on= 'away_team', right_on='team', suffixes=['_home','_away'], how='left').fillna(0) )
    X_input.drop(['team_home', 'team_away'], axis=1, inplace=True)

    X_input = encode_variables(X_input)
    # Results with CatBoost
    preds = model.predict_proba(X_input)
    # Predictions of the results
    group_df['away'] = pd.DataFrame(preds)[0]
    group_df['draw'] = pd.DataFrame(preds)[1]
    group_df['home'] = pd.DataFrame(preds)[2]

    print(group_df)

df_predictions = pd.DataFrame()

for group in groups:
    # Make predictions
    prediction = make_predictions(group)
    # Add prediction dataframe to df_predictions
    df_predictions = pd.concat([df_predictions, prediction])

# Save predictions

df_predictions.to_parquet(product['predictions'])

     home_team    away_team      away      draw      home
0  New Zealand       Norway  0.815434  0.167203  0.017363
1  Philippines  Switzerland  0.060202  0.008959  0.930839
2  New Zealand  Philippines  0.000416  0.013989  0.985596
3  Switzerland       Norway  0.816193  0.181277  0.002531
     home_team    away_team      away      draw      home
0       Zambia       Norway  0.994272  0.003989  0.001739
1  Philippines  Switzerland  0.060202  0.008959  0.930839
2       Zambia  Philippines  0.003234  0.000127  0.996638
3  Switzerland       Norway  0.816193  0.181277  0.002531
